In [1]:
#IMDB scraper

In [23]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [24]:
#quick function to get text from between brackets
#turns out I could have just used .string ><
def findtext(w):
    w = str(w)
    w = w[w.find('>')+1:w.find('<', w.find('>'), len(w))]
    return w

#quick function to get money - does not convert to string at this time
def findmoney(w):
    w = str(w)
    w = w[w.find('$')+1:w.find('<', w.find('$'), len(w))]
    return w

In [81]:
###THIS IS THE FULL SCRAPE OF A PAGE###

def Get_Movie(url):
    # Request the page and use BeautifulSoup to extract the contents
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get title
    title = soup.find(attrs={'data-testid':'hero-title-block__title'})
    title = findtext(title)

    #get release date and rating (PG, PG13, etc)
    DR = soup.find_all(attrs={'class':'TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex'})
    if DR is not None:
        if len(DR)==2:
            date = findtext(DR[0])
            rating = findtext(DR[1])
        else:
            date = findtext(DR[0])
            rating = None
    else:
        date = None
        rating = None

    #get a list of genres
    gen = soup.find(attrs={'data-testid':'storyline-genres'}) 
    if gen is not None:
        gen = gen.find_all('a')
        genres =''
        for t in range(len(gen)):
            genres = genres + findtext(gen[t]) + ' '
    else:
        genres = None

    #get box office info
    #budget often has '(estimated)' in it
    box = soup.find(attrs={'data-testid':'title-boxoffice-section'})
    if box is not None:
        budget = box.find(attrs={'data-testid':'title-boxoffice-budget'}) 
        budget = str(budget)
        budget = findmoney(budget)
        dom = box.find(attrs={'data-testid':'title-boxoffice-grossdomestic'})
        dom = findmoney(dom)
        ww = soup.find(attrs={'data-testid':'title-boxoffice-cumulativeworldwidegross'}) 
        ww = findmoney(ww)
    else:
        budget = None
        dom = None
        ww = None

    #get the description
    story = soup.find(attrs={'data-testid':'Storyline'}) 
    desc = story.find(attrs={'class':'ipc-html-content ipc-html-content--base'})
    desc = desc.div
    desc = findtext(desc)

    #get top tags
    #only shows top few tags, usually list tag is to full list which can be in hundreds
    kw = story.find(attrs={'data-testid':'storyline-plot-keywords'}) 
    if kw is not None:
        kw = kw.find_all('span')
        keywords = ''
        for t in range(len(kw)):
            #keywords.append(findtext(kw[t]))
            keywords = keywords + findtext(kw[t]) + ' '
        #if 'more' in keywords[-1]:
            #keywords = keywords[:-1]
    else:
        keywords = None

    #get runtime in minutes
    rt = soup.find(attrs={'data-testid':'title-techspec_runtime'}) 
    if rt is not None:
        rt = rt.find('div')
        hrs = findtext(rt)
        rt = str(rt)
        mins = rt[::-1]
        mins = mins[7:]
        mins = mins[mins.find('<')+1:mins.find('>', mins.find('<'), len(mins))]
        runtime = int(hrs)*60 + int(mins)
    else:
        runtime = None

    #put all that info in a list and return it 
    #NOTE: genres and keywords are both still list objects
    obs = [title,date,rating, genres, budget,dom,ww,desc,keywords, runtime]
    
    return obs

In [82]:
#ok now pull in our list of movies
movies = pd.read_csv(r'C:\Users\maria\OneDrive\Documents\AIPI510\BooksToMoviesML\FilmList.csv')
movies = movies['Link']
#drop the first one because it has not been released yet
movies = movies.drop([0])
#print(movies.isnull().sum())


#instantiate dataframe
data = pd.DataFrame(columns = ['Title', 'Release_Date', 'ViewRating', 'Genres', 'Budget', 'DomesticGross', 
                              'WorldwideGross', 'Description', 'Keywords', 'Runtime'])

In [83]:
#scrape provided pages
for this in movies:
    print(this)
    obs = Get_Movie('https://www.imdb.com{}'.format(this))
    data.loc[len(data)] = obs
        

/title/tt1160419/
/title/tt13069986/
/title/tt3228774/
/title/tt6654210/
/title/tt8150814/
/title/tt6566576/
/title/tt8893974/
/title/tt10293406/
/title/tt13056008/
/title/tt3215824/
/title/tt2076822/
/title/tt9701940/
/title/tt10521144/
/title/tt7740496/
/title/tt9100054/
/title/tt10962368/
/title/tt4479380/
/title/tt9130508/
/title/tt9701942/
/title/tt6111574/
/title/tt11657662/
/title/tt1893273/
/title/tt0499097/
/title/tt9471660/
/title/tt8482584/
/title/tt12229370/
/title/tt6432466/
/title/tt12783454/
/title/tt6571548/
/title/tt11698662/
/title/tt10505316/
/title/tt10451312/
/title/tt3696180/
/title/tt12851396/
/title/tt12299824/
/title/tt8521736/
/title/tt9110904/
/title/tt15204288/
/title/tt13304948/
/title/tt12847812/
/title/tt11540284/
/title/tt11777920/
/title/tt3301314/
/title/tt11681056/
/title/tt14599938/
/title/tt5738280/
/title/tt10644372/
/title/tt11636880/
/title/tt3804810/
/title/tt13387422/
/title/tt13873302/
/title/tt13269616/
/title/tt10510280/
/title/tt5280052/
/t

AttributeError: 'NoneType' object has no attribute 'div'

In [84]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 69
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           70 non-null     object
 1   Release_Date    70 non-null     object
 2   ViewRating      44 non-null     object
 3   Genres          70 non-null     object
 4   Budget          24 non-null     object
 5   DomesticGross   24 non-null     object
 6   WorldwideGross  24 non-null     object
 7   Description     70 non-null     object
 8   Keywords        70 non-null     object
 9   Runtime         67 non-null     object
dtypes: object(10)
memory usage: 6.0+ KB
None


In [59]:
data.


In [73]:
page = requests.get('https://www.imdb.com/title/tt9701940/')
soup = BeautifulSoup(page.content, 'html.parser')


In [74]:
DR = soup.find_all(attrs={'class':'TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex'})
#print(type(DR))
print(DR)

[<span class="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex">2021</span>, <span class="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex">R</span>]
